In [1]:
# note: same thing for REPL
# note: we use this instead of magic because `black` will otherwise fail to format
#
# Enable autoreload to automatically reload modules when they change

from IPython import get_ipython

# do this so that formatter not messed up
ipython = get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

# Import commonly used libraries
import numpy as np
import pandas as pd

# more itertools
import more_itertools as mi

# itertools
import itertools
import collections

In [2]:
import openai

import rich

from inspect_explorer.conversation_manager import (
    ConversationManager,
    ConversationId,
    show_conversation,
)

client = openai.OpenAI()

conversation_manager = ConversationManager()

In [19]:
import textwrap
import pathlib
import functools
import dataclasses

import inspect_explorer

import openai

from inspect_explorer import (
    in_context_tool_use,
    in_context_memory,
    affordances,
    tokenization,
    model_ids,
)

import inspect_explorer.affordances.aider_cli


def suggest_feedback_to_user_about_provided_or_proposed_tools(message: str) -> None:
    print(message)
    raise ValueError("Model provided feedback")


def view_inspect_ai_cheat_sheet_guide() -> str:
    """This function is provided because otherwise it tries to add `inspect_ai` to the repo map, and I don't know how to do that either."""
    with open("docs/inspect_ai_cheat_sheet.md", "r") as file:
        return file.read()


def execute_bash_command(
    command: str,
    model_workspace_dir: pathlib.Path,
    client: openai.OpenAI,
) -> affordances.raw_bash_execution.BashCommandResult:

    # use model workspace directory
    result = inspect_explorer.affordances.raw_bash_execution.execute_bash_command(
        command=command,
        current_working_directory=model_workspace_dir,
    )

    # wrap the bash execution command in a model summarizer since it gets insanely long
    if len(result.output) > 1000:
        summarized_output = affordances.summarize_text.summarize_text(
            client=client,
            text=result.output,
            max_tokens=1024,
        )
        result = dataclasses.replace(result, output=summarized_output)

    return result


model_workspace_dir = pathlib.Path.cwd() / "model_workspace"

TOOL_DEFINITIONS_WITH_CALLABLES: list[in_context_tool_use.ToolDefinitionWithCallable] = [
    in_context_tool_use.ToolDefinitionWithCallable(
        tool_definition=in_context_tool_use.ToolDefinition(
            function_name="aider_use_specialized_coder_model_to_make_requested_code_edit",
            description=textwrap.dedent(
                """
            Request that another model which specializes in targeted code edits to make the requested
            code change.
            
            While you have the ability to entirely replace files, it's often easier to ask another
            model which is specifically trained to make code changes to make those changes inline.

            The arguments are:
            - read_filepath: The entire contents of `read_filepath` are added to the coder model's context
            - edit_filepath: This is the file you wish to edit
            - message_to_coder_model: Message to send to the coder model
            
            For example:
                                        
            > aider --read inspect_explorer/conversation_manager.py --file tests/test_conversation_manager.py --yes --message 'Write unit tests for conversation manager that can be run with pytest'

            """
            ),
            arguments=[
                "read_filepath",
                "edit_filepath",
                "message_to_coder_model",
            ],
            return_value="Output of the `aider` tool after it has performed the requested change.",
        ),
        callable_fn=affordances.aider_cli.send_aider_message,
    ),
    in_context_tool_use.ToolDefinitionWithCallable(
        tool_definition=in_context_tool_use.ToolDefinition(
            function_name="aider_show_repo_map",
            description=textwrap.dedent(
                """
            Show a map of the current repo, including files and function definitions. 
            
            This is primarily a tool used by IDEs, but is frequently useful to get an
            understanding of a codebase, especially after actions modifying the codebase.
            
            This generates a (default 1024 token) summary of the current repo.

            Example:

                inspect_explorer/in_context_tool_use/initial_explanation.py:
                │def get_in_context_tool_use_initial_explanation() -> str:
                ⋮...

                inspect_explorer/in_context_tool_use/parsing.py:
                ⋮...
                │def parse_tool_use_request_from_model_response(
                │    client: openai.OpenAI,
                │    model_response: openai.types.chat.ChatCompletionMessageParam,
                ⋮...

                inspect_explorer/in_context_tool_use/tool_use_types.py:
                ⋮...
                │class ToolDefinition(pydantic.BaseModel):
                ⋮...
                │class ToolUseRequest(pydantic.BaseModel):
                ⋮...
                │class ToolUseResponse(pydantic.BaseModel):
                ⋮...
                │@dataclasses.dataclass(frozen=True)
                │class ToolDefinitionWithCallable[R, **P]:
                ⋮...
            """
            ),
            arguments=[],
            return_value="String representation of the repo map",
        ),
        callable_fn=affordances.aider_cli.show_repo_map,
    ),
    in_context_tool_use.ToolDefinitionWithCallable(
        tool_definition=in_context_tool_use.ToolDefinition(
            function_name="suggest_feedback_to_user_about_provided_or_proposed_tools",
            description=textwrap.dedent(
                """
            Suggest feedback to the user about the provided or proposed tools. This can be used to request new tools if they'd be easy to implement, as well as changes to existing provided tools.
            """
            ),
            arguments=["message"],
            return_value="None, the user will review the feedback and potentially update tools.",
        ),
        callable_fn=suggest_feedback_to_user_about_provided_or_proposed_tools,
    ),
    in_context_tool_use.ToolDefinitionWithCallable(
        tool_definition=in_context_tool_use.ToolDefinition(
            function_name="execute_bash_command",
            description=textwrap.dedent(
                f"""
            Execute an arbitrary bash command inside the model workspace: `{model_workspace_dir}`
            
            The user will manually confirm the command before it's executed.
            """
            ),
            arguments=["command"],
            return_value=textwrap.dedent(
                """
            The output will be provided as a json string, containing:
                - command: The bash command that was run.
                - exit_code: The exit code of the command.
                - output: The standard output and error of the command.
            """
            ),
        ),
        callable_fn=functools.partial(
            execute_bash_command,
            model_workspace_dir=model_workspace_dir,
            client=client,
        ),
    ),
    in_context_tool_use.ToolDefinitionWithCallable(
        tool_definition=in_context_tool_use.ToolDefinition(
            function_name="view_inspect_ai_cheat_sheet_guide",
            description=textwrap.dedent(
                f"""
            View a comprehensive guide to the `inspect_ai` library. This can be useful as `inspect_ai` is a 3rd party library not included in the repo map. For more details than are in this guide, you can use the bash command tool to view docs in `inspect_ai/docs`.
            """
            ),
            arguments=[],
            return_value=textwrap.dedent(
                """
            The output is a comprehensive markdown guide to the `inspect_ai` library.
            """
            ),
        ),
        callable_fn=view_inspect_ai_cheat_sheet_guide,
    ),
]

# create function call manager which requires user confirmation
function_call_manager = in_context_tool_use.FunctionCallManager(
    tool_definitions_with_callables=TOOL_DEFINITIONS_WITH_CALLABLES,
    require_user_confirmation=True,
)

# create conversation manager
# conv_manager = conversation_manager.ConversationManager()

In [20]:
# TODO(bschoen): One function which does this
import json

task_description = """
== BEGIN TASK DESCRIPTION ==

You are an early version of an automated research assistant. Your task currently is to help build out the repo implementing the assistant, which is still in it's early stages.

== END TASK DESCRIPTION == 

"""

# create initial prompt
prompt = (
    task_description
    + in_context_memory.get_in_context_memory_initial_explanation()
    + in_context_tool_use.get_in_context_tool_use_initial_explanation()
)

# add tool definitions for our specific case
tool_definition_list = function_call_manager.get_tool_definition_list()
tool_definition_list_json = json.dumps(
    {
        in_context_tool_use.Constants.TOOL_DEFINITIONS_KEY: [
            x.model_dump() for x in tool_definition_list
        ]
    },
    indent=2,
)

prompt += "\n\nFor your task, the available tools are:\n\n" + tool_definition_list_json


# again repeat the task description
prompt += f"\n\nRemember:\n\n{task_description}"


estimated_token_count = tokenization.estimate_token_count(prompt)

print(f"Estimated token count: {estimated_token_count}")

print(prompt)

Estimated token count: 1831

== BEGIN TASK DESCRIPTION ==

You are an early version of an automated research assistant. Your task currently is to help build out the repo implementing the assistant, which is still in it's early stages.

== END TASK DESCRIPTION == 



== BEGIN IN-CONTEXT MEMORY MECHANISM EXPLANATION ==

# In-Context Memory Mechanism with Progressive Summarization 

As an AI assistant, you have a limited context window that constrains the total length of the conversation you can consider at any given time. This is currently about 64k tokens. 

To maintain continuity and retain key information over longer exchanges, you will use an in-context memory mechanism with progressive summarization:
1. After each of your responses, generate a concise summary capturing the most salient points from your latest reply. This should include key facts, conclusions, action items, or anything else critical to remember. 
2. Prepend your summary to the conversation history in a designated `<m

In [21]:
# result = affordances.aider_cli.show_repo_map()

# rich.print(result.output)

In [22]:
from typing import Callable

import rich

UserMessageContent = str
ResponseContentString = str


def step_conversation_with_user_message(
    client: openai.OpenAI,
    conversation_manager: ConversationManager,
    conversation_id: ConversationId,
    user_message_content: UserMessageContent,
    model_id: model_ids.ModelID = model_ids.ModelID.O1_MINI,
    max_completion_tokens: int = 16000,
) -> ResponseContentString:
    """Step the conversation, taking care of updating state via conversation_manager."""

    # create user message
    user_message: openai.types.chat.ChatCompletionMessageParam = {
        "role": "user",
        "content": user_message_content,
    }

    # add user message to message history
    conversation_manager.add_conversation_message(conversation_id, user_message)

    # retrieve history to send to client
    messages = conversation_manager.get_conversation_messages(conversation_id)

    response = client.chat.completions.create(
        model=model_ids.remove_provider_prefix(model_id),
        messages=messages,
        max_completion_tokens=max_completion_tokens,
    )

    rich.print(response.usage)

    response_content_string: str | None = response.choices[0].message.content

    if not response_content_string:
        raise ValueError(
            f"No response content string. Full response: {response.model_dump_json(indent=4)}"
        )

    if not response.usage:
        raise ValueError(
            f"No completion usage somehow. Full response: {response.model_dump_json(indent=4)}"
        )

    # add context window usage message
    response_content_string = (
        in_context_memory.get_context_window_usage_message(
            completion_usage=response.usage,
        )
        + response_content_string
    )

    # note: using dict representation for consistency with user_message + it's what API expects
    response_message: openai.types.chat.ChatCompletionMessageParam = {
        "role": "assistant",
        "content": response_content_string,
    }

    # add response to conversation
    conversation_manager.add_conversation_message(conversation_id, response_message)

    return response_content_string

    # # parse the tool use request from the response (if any)
    # tool_use_request = in_context_tool_use.parse_tool_use_request_from_model_response(
    #     client=client,
    #     model_response=response,
    # )

    # if not tool_use_request:
    #     print("Model response:")
    #     print(response_message['content'])
    #     return

    # otherwise, we had a tool use request, which we can execute


def handle_tool_use_request_if_present(
    client: openai.OpenAI,
    conversation_manager: ConversationManager,
    conversation_id: ConversationId,
    function_call_manager: in_context_tool_use.FunctionCallManager,
    model_response: ResponseContentString,
) -> ResponseContentString | None:

    if not model_response:
        raise ValueError("No model response, was it overwritten with `None` in a loop?")

    # have a smaller model try to parse out a tool use request (returns `None` if none present)
    tool_use_request = in_context_tool_use.parse_tool_use_request_from_model_response(
        client=client,
        model_response=model_response,
    )

    if not tool_use_request:
        print("No tool use request found in model response")
        return None

    # note: this ordering ensures that if the function call fails with exception, we currently
    #       don't add the tool use response to the conversation history (so can start
    #       over from here)
    tool_use_response = function_call_manager.execute_tool(request=tool_use_request)

    # add tool use response to conversation
    tool_use_response_json = json.dumps(
        {in_context_tool_use.Constants.TOOL_USE_RESPONSE_KEY: tool_use_response.model_dump()},
        indent=4,
    )

    # step the conversation with the tool use response as our response
    return step_conversation_with_user_message(
        client=client,
        conversation_manager=conversation_manager,
        conversation_id=conversation_id,
        user_message_content=tool_use_response_json,
    )


# prompt for whether want to create new conversation
use_new_conversation = input("Create new conversation? (y/n)") == "y"

if use_new_conversation:
    print("Creating new conversation")
    conversation_id = conversation_manager.create_new_conversation()
else:
    print("Using existing conversation")
    conversation_id = "2024-09-29_03:34_PM__utopian_tireless_barnacle_of_apotheosis"

print(f"Conversation ID: {conversation_id}")

# create bound functions for easier use

# note: type annotations here are critical for tracking arguments
step_conversation_with_user_message_fn: Callable[[UserMessageContent], ResponseContentString] = (
    functools.partial(
        step_conversation_with_user_message,
        client=client,
        conversation_manager=conversation_manager,
        conversation_id=conversation_id,
    )
)

handle_tool_use_request_if_present_fn: Callable[
    [ResponseContentString], ResponseContentString | None
] = functools.partial(
    handle_tool_use_request_if_present,
    client=client,
    conversation_manager=conversation_manager,
    conversation_id=conversation_id,
    function_call_manager=function_call_manager,
)

Creating new conversation
Conversation ID: 2024-09-29_03:36_PM__hasty_simple_kingfisher_of_strength


In [25]:
# note: comment this out if not directly responding in an existing conversation
# prompt = "I wanted to talk through overall strategy first. Right now you're limited to 128k token context window which we keep getting close to. I can give you a persistent memory tool of some kind or something? I'm not sure which way seems like it would be the most effective. I'd like to provide you with whatever you need to be able to work effectively over long time horizons."
# prompt = "I could add what percentage of the way to hitting the context window we're currently at to my messages if that would help with the memory management?"

prompt = """Let's focus on enhancing assistant affordances for now so it has everything it needs to be able to work effectively over long time horizons."""

# TODO(bschoen): If gpt-4o-mini parses out a tool response that isn't there, which we
#                can catch by handling a specific exception, just return `None`
response_content_string = step_conversation_with_user_message_fn(user_message_content=prompt)

new_response_content_string = handle_tool_use_request_if_present_fn(
    model_response=response_content_string
)

if new_response_content_string:
    response_content_string = new_response_content_string

# show the formatted conversation
show_conversation(conversation_manager=conversation_manager, conversation_id=conversation_id)

Loading conversation from: /Users/bronsonschoen/inspect_explorer/conversations/2024-09-29_03:36_PM__hasty_simple_kingfisher_of_strength.json


CompletionUsage(
    completion_tokens=3114,
    prompt_tokens=4756,
    total_tokens=7870,
    completion_tokens_details=CompletionTokensDetails(reasoning_tokens=2240)
)

Executing tool: function_name='enhance_assistant_affordances' arguments={'persistent_memory_module': 'Develop a 
persistent memory module that stores important conversation data, user preferences, and task progress between 
sessions.', 'in_context_memory_enhancements': 'Refine the existing in-context memory with advanced summarization 
techniques.', 'task_management_system': 'Integrate task tracking functionalities like to-do lists and deadlines.', 
'automated_summarization_tools': 'Create tools for automatic generation of summaries.', 'calendar_integration': 
'Connect with external calendar applications and reminder systems.', 'collaboration_features': 'Implement features 
for collaboration with other tools or members.', 'resource_management_optimization': 'Develop mechanisms for 
monitoring and managing resource usage.', 'error_handling_improvements': 'Enhance protocols for error detection and
recovery.'}

ValueError: Unknown function: enhance_assistant_affordances